In [ ]:
import sys
import numpy as np
from imageio import imread, imwrite
from tqdm import trange
import cv2 

Compute energy using |∂I/∂x| + |∂I/∂y| without predefined functions

In [ ]:
def calc_energy(img):
    gray = 0.299 * img[:, :, 0] + 0.587 * img[:, :, 1] + 0.114 * img[:, :, 2]
    dx = np.zeros_like(gray)
    dy = np.zeros_like(gray)

    # Compute gradients
    dx[:, 1:-1] = gray[:, 2:] - gray[:, :-2]
    dy[1:-1, :] = gray[2:, :] - gray[:-2, :]

    return np.abs(dx) + np.abs(dy)

Finds the minimum energy seam using dynamic programming

In [ ]:
def find_seam(img):
    r, c = img.shape[:2]
    energy = calc_energy(img)
    M = energy.copy()
    backtrack = np.zeros_like(M, dtype=np.int32)

    for i in range(1, r):
        for j in range(c):
            left = M[i-1, j-1] if j > 0 else float('inf')
            up = M[i-1, j]
            right = M[i-1, j+1] if j < c-1 else float('inf')

            min_val = min(left, up, right)
            M[i, j] += min_val

            if min_val == left:
                backtrack[i, j] = j-1
            elif min_val == up:
                backtrack[i, j] = j
            else:
                backtrack[i, j] = j+1

    # Backtrack to find the seam
    seam = np.zeros(r, dtype=np.int32)
    seam[-1] = np.argmin(M[-1])
    for i in range(r-2, -1, -1):
        seam[i] = backtrack[i+1, seam[i+1]]

    return seam

In [10]:
def remove_seam(img, seam):
    """Removes the given seam from the image."""
    r, c, _ = img.shape
    mask = np.ones((r, c), dtype=np.bool_)
    
    for i in range(r):
        mask[i, seam[i]] = False
    
    img = img[mask].reshape((r, c-1, 3))
    return img

def visualize_seams(img, seams):
    """Visualizes removed seams by drawing them on the original image."""
    img_vis = img.copy()
    for seam in seams:
        for i in range(img.shape[0]):
            img_vis[i, seam[i]] = [0, 0, 255]  # Mark seam in red
    return img_vis

def seam_carving(img, scale, mode):
    """Performs seam carving to resize an image."""
    r, c, _ = img.shape
    target_c = int(scale * c) if mode == 'c' else c
    target_r = int(scale * r) if mode == 'r' else r

    seams = []
    
    if mode == 'c':
        for _ in trange(c - target_c):
            seam = find_seam(img)
            seams.append(seam)
            img = remove_seam(img, seam)

    elif mode == 'r':
        img = np.rot90(img, 1, (0, 1))  # Rotate to treat rows as columns
        for _ in trange(r - target_r):
            seam = find_seam(img)
            seams.append(seam)
            img = remove_seam(img, seam)
        img = np.rot90(img, 3, (0, 1))  # Rotate back

    return img, visualize_seams(imread(sys.argv[3]), seams)

def main():
    if len(sys.argv) != 5:
        print('Usage: python carver.py <r/c> <scale> <image_in> <image_out>', file=sys.stderr)
        sys.exit(1)

    mode = sys.argv[1]
    scale = float(sys.argv[2])
    input_path = sys.argv[3]
    output_path = sys.argv[4]

    img = imread(input_path)
    resized_img, seam_visualization = seam_carving(img, scale, mode)

    imwrite(output_path, resized_img)
    imwrite(f"seams_{output_path}", seam_visualization)



if __name__ == "__main__":
    main()


C:\Users\shahd\AppData\Local\Temp\ipykernel_12300\240576568.py:103: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(input_path)
100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [01:10<00:00,  3.37it/s]
C:\Users\shahd\AppData\Local\Temp\ipykernel_12300\240576568.py:91: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return img, visualize_seams(imread(sys.argv[3]), seams)
